In [1]:
# Cargamos librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Carga desde un archivo .xlsx sin indice
Credicel = pd.read_excel('BaseLimpia.xlsx')
Credicel.head(5)

,folio,folio_solicitud,precio,enganche,descuento,monto_accesorios,status,fraude,inversion,pagos_realizados,...,fecha_ultimo_pago,fecha_pago_proximo,status_cuenta,razones_buro,cp_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,30,1949.0,780.0,0.0,0.0,2,0,0,0,...,Desconocido,Desconocido,Aun no se determina status,Desconocido,Desconocido,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02
1,4,33,2999.0,1050.0,0.0,0.0,2,0,0,0,...,Desconocido,Desconocido,Aun no se determina status,Desconocido,Desconocido,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07
2,5,37,1959.0,490.0,0.0,0.0,3,0,0,0,...,Desconocido,Desconocido,Cancelado,Desconocido,Desconocido,TANTOYUCA,VER.,0,0,0
3,6,38,1959.0,490.0,0.0,0.0,2,0,0,0,...,Desconocido,Desconocido,Aun no se determina status,Desconocido,Desconocido,AMXTLAN PUE,Desconocido,Zacatlan,Puebla,SAMJ790115MPLNRS02
4,7,44,4299.0,1075.0,0.0,0.0,1,0,0,0,...,2022-03-05 00:00:00,2022-03-12 00:00:00,Mora,Desconocido,Desconocido,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07


In [3]:
# Definir los límites de categorización
bins = [-65, 0, 103]  # Los límites son -64 a 0 y 1 a 103

# Definir los nombres de las categorías
labels = ['No hay Riesgo', 'Hay Riesgo']

In [4]:
# Aplicar la categorización utilizando cut en la columna "Edades"
Credicel['riesgo_cat'] = pd.cut(Credicel['riesgo'], bins=bins, labels=labels)

In [5]:
Credicel['riesgo_cat']

0        No hay Riesgo
1        No hay Riesgo
2        No hay Riesgo
3        No hay Riesgo
4        No hay Riesgo
             ...      
22730       Hay Riesgo
22731       Hay Riesgo
22732       Hay Riesgo
22733    No hay Riesgo
22734       Hay Riesgo
Name: riesgo_cat, Length: 22735, dtype: category
Categories (2, object): ['No hay Riesgo' < 'Hay Riesgo']

In [6]:
CredicelComp = Credicel[['riesgo', 'riesgo_cat']]
CredicelComp

,riesgo,riesgo_cat
0,0.00,No hay Riesgo
1,0.00,No hay Riesgo
2,0.00,No hay Riesgo
3,0.00,No hay Riesgo
4,0.00,No hay Riesgo
...,...,...
22730,27.40,Hay Riesgo
22731,19.00,Hay Riesgo
22732,26.65,Hay Riesgo
22733,-17.35,No hay Riesgo


In [7]:
Credicel

,folio,folio_solicitud,precio,enganche,descuento,monto_accesorios,status,fraude,inversion,pagos_realizados,...,fecha_pago_proximo,status_cuenta,razones_buro,cp_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp,riesgo_cat
0,3,30,1949.0,780.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02,No hay Riesgo
1,4,33,2999.0,1050.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07,No hay Riesgo
2,5,37,1959.0,490.0,0.0,0.0,3,0,0,0,...,Desconocido,Cancelado,Desconocido,Desconocido,TANTOYUCA,VER.,0,0,0,No hay Riesgo
3,6,38,1959.0,490.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,AMXTLAN PUE,Desconocido,Zacatlan,Puebla,SAMJ790115MPLNRS02,No hay Riesgo
4,7,44,4299.0,1075.0,0.0,0.0,1,0,0,0,...,2022-03-12 00:00:00,Mora,Desconocido,Desconocido,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07,No hay Riesgo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,96502,3349.0,297.0,0.0,0.0,1,0,0,0,...,2023-04-25 00:00:00,Aun no se determina status,"K0,J0,D8,E2",97880,OXKUTZCAB,YUC,Oxkutzcab,Yucatan,PACT990804MYNCCH05,Hay Riesgo
22731,22968,96479,4100.0,1209.0,0.0,0.0,1,0,0,0,...,2023-04-25 00:00:00,Aun no se determina status,"K0,J0,D8,E2",37295,LEON,GTO,Leon,Guanajuato,HEGB950620MGTRMN03,Hay Riesgo
22732,22969,96442,4849.0,1152.0,0.0,0.0,1,0,0,0,...,2023-04-25 00:00:00,Aun no se determina status,"K0,D7,F7,D8",20210,AGUASCALIENTES,AGS,Aguascalientes,Aguascalientes,GADF870205HASLZL07,Hay Riesgo
22733,22970,96536,9799.0,2799.0,0.0,0.0,1,0,0,0,...,2023-04-25 00:00:00,Aun no se determina status,"J0,G1,E0,E2",78434,SOLEDAD DE GRACIANO SANCHEZ,SLP,Soledad De Graciano Sanchez,San Luis Potosi,DELJ700417HSPLRS03,No hay Riesgo


In [8]:
#Corroboramos valores nulos
ValoresNulos = Credicel.isnull().sum()
ValoresNulos

folio                 0
folio_solicitud       0
precio                0
enganche              0
descuento             0
monto_accesorios      0
status                0
fraude                0
inversion             0
pagos_realizados      0
reautorizacion        0
puntos                0
riesgo                0
score_buro            0
porc_eng              0
limite_credito        0
semana_actual         0
edad_cliente          0
costo_total           0
monto_financiado      0
semana                0
tag                   0
fecha                 0
marca                 0
modelo                0
plazo                 0
agente_venta          0
dis_venta             0
empresa               0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
razones_buro          0
cp_cliente            0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
riesgo_cat            0
dtype: int64

In [9]:
frecuencia = Credicel["riesgo_cat"].value_counts()

frecuencia

riesgo_cat
Hay Riesgo       12038
No hay Riesgo    10697
Name: count, dtype: int64

In [12]:
#Credicel.to_excel("CredicelTodo.xlsx")

In [11]:
#Credicel.to_csv("CredicelCSV.csv")